Zaimplementuj aplikację szacującą czas ukończenia półmaratonu dla zadanych danych

1. Umieść dane w Digital Ocean Spaces

1. Napisz notebook, który będzie Twoim pipelinem do trenowania modelu
    * czyta dane z Digital Ocean Spaces
    * czyści je
    * trenuje model (dobierz odpowiednie metryki [feature selection])
    * nowa wersja modelu jest zapisywana lokalnie i do Digital Ocean Spaces

1. Aplikacja
    * opakuj model w aplikację streamlit
    * wdróż (deploy) aplikację za pomocą Digital Ocean AppPlatform 
    * wejściem jest pole tekstowe, w którym użytkownik się przedstawia, mówi o tym
    jaka jest jego płeć, wiek i tempo na 5km
    * jeśli użytkownik podał za mało danych, wyświetl informację o tym jakich danych brakuje
    * za pomocą LLM (OpenAI) wyłuskaj potrzebne dane, potrzebne dla Twojego modelu
    do określenia, do słownika (dictionary lub JSON)
    * tę część podepnij do Langfuse, aby zbierać metryki o skuteczności działania LLM'a



In [ ]:
# Importowanie bibliotek
import pandas as pd
from pycaret.regression import *

# Wczytanie danych
df_2023 = pd.read_csv('halfmarathon_wroclaw_2023__final.csv', sep=';')
df_2024 = pd.read_csv('halfmarathon_wroclaw_2024__final.csv', sep=';')

# Połączenie obu DataFrame w jeden
df = pd.concat([df_2023, df_2024], ignore_index=True)

# Kolumny do usunięcia
columns_to_drop = ['Miejsce', 'Kategoria wiekowa', 'Numer startowy', 'Imię', 'Nazwisko', 'Miasto', 'Kraj', 'Drużyna', 'Płeć Miejsce', 'Kategoria wiekowa Miejsce', 
                  '5 km Miejsce Open', '10 km Miejsce Open', '15 km Miejsce Open', '20 km Miejsce Open', 'Tempo Stabilność']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Usunięcie wierszy z brakującymi danymi w istotnych kolumnach
df = df[df['5 km Czas'].notna()]
df = df[df['10 km Czas'].notna()]
df = df[df['15 km Czas'].notna()]
df = df[df['20 km Czas'].notna()]
df = df[df['Czas'].notna()]
df = df[df['Rocznik'].notna()]

# Funkcje konwertujące czas i tempo na sekundy
def convert_time_to_seconds(time_str):
    if pd.isna(time_str): return None
    time_parts = time_str.split(':')
    if len(time_parts) == 2:
        minutes, seconds = map(int, time_parts)
        return minutes * 60 + seconds
    elif len(time_parts) == 3:
        hours, minutes, seconds = map(int, time_parts)
        return hours * 3600 + minutes * 60 + seconds
    return None

def convert_pace_to_seconds_per_km(pace):
    if pd.isna(pace): return None
    if isinstance(pace, float):  # Already in decimal minutes
        minutes = int(pace)
        seconds = (pace - minutes) * 60
        return round(minutes * 60 + seconds)
    try:
        minutes, seconds = map(int, pace.split(':'))
        return minutes * 60 + seconds
    except ValueError:
        return None

# Zastosowanie konwersji
df['Czas półmaratonu (sekundy)'] = df['Czas'].apply(convert_time_to_seconds)
df['Czas na 5km (sekundy)'] = df['5 km Czas'].apply(convert_time_to_seconds)
df['Czas na 10km (sekundy)'] = df['10 km Czas'].apply(convert_time_to_seconds)
df['Czas na 15km (sekundy)'] = df['15 km Czas'].apply(convert_time_to_seconds)
df['Czas na 20km (sekundy)'] = df['20 km Czas'].apply(convert_time_to_seconds)

df['Tempo na 5 km (sekundy na km)'] = df['5 km Tempo'].apply(convert_pace_to_seconds_per_km)
df['Tempo na 10 km (sekundy na km)'] = df['10 km Tempo'].apply(convert_pace_to_seconds_per_km)
df['Tempo na 15 km (sekundy na km)'] = df['15 km Tempo'].apply(convert_pace_to_seconds_per_km)
df['Tempo na 20 km (sekundy na km)'] = df['20 km Tempo'].apply(convert_pace_to_seconds_per_km)
df['Tempo półmaratonu (sekundy)'] = df['Tempo'].apply(convert_pace_to_seconds_per_km)

# Usunięcie niepotrzebnych kolumn
df.drop(columns=['Czas', 'Tempo', '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo'], inplace=True, errors='ignore')

# Mapowanie płci
df['Płeć'] = df['Płeć'].map({'K': 0, 'M': 1})

# Ujednolicenie nazw kolumn
df.columns = df.columns.str.replace(r'\s+', '_', regex=True).str.replace(r'[^\w]', '', regex=True)

# Sprawdzenie wymaganych kolumn
required_columns = ['Rocznik', 'Płeć', 'Czas_na_15km_sekundy', 'Czas_półmaratonu_sekundy']
for col in required_columns:
    if col not in df.columns:
        df[col] = 0  # Dodanie kolumn, których brakuje

# Wybór istotnych kolumn do trenowania modelu
df = df[['Rocznik', 'Płeć', 'Czas_na_15km_sekundy', 'Czas_półmaratonu_sekundy']]

# Usunięcie wierszy z brakującymi wartościami
df = df.dropna()

# Podział danych na zbiór treningowy i testowy
train_data, test_data = train_test_split(df, test_size=0.2, random_state=123)

# Ustawienie środowiska PyCaret
setup(data=train_data, target='Czas_półmaratonu_sekundy', session_id=123, normalize=True, feature_selection=False, remove_multicollinearity=False)

# Porównanie modeli
best_model = compare_models(fold=5)

# Finalizacja najlepszego modelu
final_model = finalize_model(best_model)

# Debugowanie: Zobaczenie struktury modelu
print(get_config('X_train').shape)
print(get_config('X_test').shape)

# Zapisanie najlepszego modelu
save_model(final_model, 'final_model')


,Description,Value
0,Session id,123
1,Target,Czas_półmaratonu_sekundy
2,Target type,Regression
3,Original data shape,"(14316, 4)"
4,Transformed data shape,"(14316, 4)"
5,Transformed train set shape,"(10021, 4)"
6,Transformed test set shape,"(4295, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,126.9148,34994.3045,186.5212,0.9764,0.0235,0.0166,0.7580
lasso,Lasso Regression,126.8633,34995.4859,186.5233,0.9764,0.0235,0.0166,0.6260
ridge,Ridge Regression,126.9114,34994.3425,186.5213,0.9764,0.0235,0.0166,0.5500
lar,Least Angle Regression,126.9148,34994.3045,186.5212,0.9764,0.0235,0.0166,0.0080
llar,Lasso Least Angle Regression,126.8635,34995.4675,186.5232,0.9764,0.0235,0.0166,0.0100
br,Bayesian Ridge,126.9145,34994.3051,186.5212,0.9764,0.0235,0.0166,0.0080
huber,Huber Regressor,125.6529,35402.5480,187.5982,0.9761,0.0236,0.0164,0.0100
gbr,Gradient Boosting Regressor,127.0886,35542.2818,188.0759,0.9760,0.0237,0.0166,0.0760
omp,Orthogonal Matching Pursuit,127.9732,35564.9599,188.0367,0.9760,0.0237,0.0167,0.0100
par,Passive Aggressive Regressor,126.2196,35748.4413,188.4970,0.9759,0.0237,0.0165,0.0100


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Rocznik', 'Płeć',
                                              'Czas_na_15km_sekundy'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_estimator', LinearRegression(n_jobs=-1))]),
 'final_model.pkl')